# espnet_onnx demonstration

This notebook provides a simple demonstration of how to export your trained model and use it for inference.

see also:
- ESPnet: https://github.com/espnet/espnet
- espnet_onnx: https://github.com/Masao-Someki/espnet_onnx

Author: [Masao Someki](https://github.com/Masao-Someki)


## Table of Contents

- Install Dependency
- Export your model
- Inference with onnx

# Install Dependency
To run this demo, you need to install the following packages.
- espnet_onnx
- torch >= 1.11.0 (already installed in Colab)
- espnet
- espnet_model_zoo
- onnx

`torch`, `espnet`, `espnet_model_zoo`, `onnx` is required to run the exportation demo.

In [ ]:
!pip install -U espnet_onnx espnet espnet_model_zoo onnx --no-cache-dir

# in this demo, we need to update scipy to avoid an error
!pip install -U scipy numpy==1.23.5 pyworld==0.3.2

And we need additional dependency `onnxruntime-gpu` to run inference on the GPU.

In [ ]:
!pip install onnxruntime-gpu

# Export your model

## Export model from espnet_model_zoo

The easiest way to export a model is to use `espnet_model_zoo`. You can download, unpack, and export the pretrained models with `export_from_pretrained` method.
`espnet_onnx` will save the onnx models into cache directory, which is `${HOME}/.cache/espnet_onnx` in default.

In [ ]:
# export the model.
from espnet_onnx.export import ASRModelExport

tag_name = 'pyf98/librispeech_conformer_hop_length160'

m = ASRModelExport()
m.set_export_config(
    max_seq_len=5000,
)
m.export_from_pretrained(
    tag_name
)

# Inference with onnxruntime
Now, let's use the exported models for inference.
Please enable the GPU resource to run the following codes.

In [ ]:
# please provide the tag_name to specify exported model.
tag_name = 'pyf98/librispeech_conformer_hop_length160'

# upload wav file and let's inference!
import librosa
from google.colab import files

wav_file = files.upload()
y, sr = librosa.load(list(wav_file.keys())[0], sr=16000)

# Use the exported onnx file to inference.
from espnet_onnx import Speech2Text

speech2text = Speech2Text(tag_name, providers=['CUDAExecutionProvider'])
nbest = speech2text(y)
print(nbest[0][0])